# Pre-Requisites

Load neccesary packages/libraries and print versions for reference.

Reference: Python 3.12.7 - miniconda3

In [ ]:
# Packages and Libraries
import cellxgene_census
import tiledbsoma
import spatialdata as sd
import spatialdata_plot

import warnings
warnings.filterwarnings("ignore")

# Versions of required libraries
print("cellxgene_census",cellxgene_census.__version__)
print("tiledbsoma", tiledbsoma.__version__)
print("spatialdata", sd.__version__)
print("spatialdata plot", spatialdata_plot.__version__)

/Users/aumchampaneri/.pyenv/versions/miniconda3-latest/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


cellxgene_census 1.16.2
tiledbsoma 1.15.6
spatialdata 0.3.0
spatialdata plot 0.2.9


# Spatial Data Export

Query the human experiment in census enabling visualization and analysis.

Perform the query using tiledbsoma instead of cellxgene_census

In [ ]:
# Export to spatial data

census = cellxgene_census.open_soma(census_version="2025-01-30")

export = census["census_spatial_sequencing"]["homo_sapiens"]

#tiledbsoma export query

with export.axis.query(
    measurement_name="RNA",
    obs_query=tiledbsoma.AxisQuery(value_filter="dataset_id == ' ") # Queries specific dataset
)

as query:
    sdata = query.to_spatialdata(X_name="raw")

sdata